<a href="https://colab.research.google.com/github/Wang-Kevin900/DataVisualization/blob/main/EsercizioReti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup libraries

In [ ]:
!pip install bokeh
!pip install altair
!pip install plotly
!pip install pandas
!pip install datetime

# Get data

Importing the various library needed to the program, importing the txt sheet where all the data are (format: date - hour - value) and lastly creating all the needed variable

In [ ]:
from google.colab import files
from datetime import datetime

# get the data from the text file
file_data = files.upload()
valori = list(file_data.values());

# prepare required variables
date_list = []
value_list = []
date_format = 10
space_between_date_hour = 1
hour_format = 8

string_of_data = ""

# Trasform Data

Rearrange all the data in sets of tuples 

In [ ]:
for i in valori[0]:
  # 13 is a value ascii that means "Carriage Return"
  if i == 13:
    pass
  # 10 is a value ascii that means "New Line"
  # as long it isn't a new line, get all the letters in the data
  elif i != 10:
    string_of_data = string_of_data + chr(i);
  # once it encountered a \n, write all the data
  else:
    date_list.append(string_of_data[0:date_format+space_between_date_hour+hour_format])
    value_list.append(string_of_data[date_format+space_between_date_hour+hour_format+1:])
    string_of_data = ""

Convert the value from string to float, also replace the "," in "." so that the casting is possible

In [ ]:
list_of_float_value = []
for item in value_list:
    value = item.replace(",", ".")
    list_of_float_value.append(float(value))

# replace the old data with the new data
value_list = list_of_float_value

Convert the value from string to datetime format


In [ ]:
from datetime import datetime

list_of_date_value = []
for item in date_list:
    list_of_date_value.append(datetime.strptime(item, '%d-%m-%Y %H:%M:%S'))

# replace the old data with the new data
date_list = list_of_date_value

Convert all the lists (date list and value list) in Pandas DataFrame, then, visualize the list created

In [ ]:
import pandas as pd

# convert the 2 lists in a pandas dataframe
data = pd.DataFrame({'date': date_list,
                     'value': value_list})

# print out the datagram
data

# Draw Chart

Draw the Chart using the Bokeh library (the reason why i used this library is because it is easily customizable and it supports the callback event, in wich though, you have to write the code in JS; there is a method to convert some data from JS data kernel to Python data kernel by using IPython, but because of google colab enviroment, wich for some reason doesn't work, it is not possible. (a solution is to install and use Jupyter Notebook))

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.tools import *


output_notebook()

source = ColumnDataSource(data)

# select the desired tool that can be used with the chart
chart_tools="pan,wheel_zoom,reset,hover,poly_select,box_select"
tooltip = HoverTool(
    tooltips=[
        ("index", "$index"),
        ("Value", "@value{0.2f}"),
        ("Date", "@date{%F}"),
    ],
    # format the date in datetime type (the default is numerical format)
    formatters={
        '@date': 'datetime'
    },
    # display a tooltip whenever the cursor is vertically in line with a data point
    mode='vline'
)


chart = figure(title='Data Visualization', plot_width=1300, plot_height=500, y_axis_label='Values', x_axis_label='Date', x_axis_type='datetime',
           tools=chart_tools, toolbar_location='above', tooltips="")
chart.add_tools(tooltip)

# draw in the chart both line and circle, so that the obtained effect is that of linked points
symbol1 = chart.line(x='date', y='value', line_width=2, source=source)
symbol2 = chart.circle(x='date', y='value', fill_color="white", size=6, source=source)

# Execute the JS code when the user select any data in the chart with the box select tool
source.selected.js_on_change("indices", CustomJS(args=dict(chart=chart, source=source), code="""
        //print the selected data in the coms of the browser (to see the data, press F12)
        console.log(source.selected.indices);
        
        //Calculus.....
        """
))

show(chart)

# Other Library used to visualize data


Test of creating the chart using various libraries...


Draw everything in a chart using the altair library (i haven't used this library since it doesn't have any support for return the data selected)

In [ ]:
import altair as alt

brush = alt.selection_interval(encodings=['x'])

color = alt.condition(brush, alt.value('blue'), alt.value('lightgray'))

chart = alt.Chart(data).mark_line(point={
    "filled": False,
    "fill": 'white'
    }).encode(
        y=alt.Y('value', sort=alt.EncodingSortField('value', order='descending')),
        x='date',
        color=color
).add_selection(
    brush
)

chart 

Draw everything in a chart using the Plotly library (i haven't used this library since it doesn't have any support for return the data selected, an exemption is DASH that is the framework built-on this library, but using this, require a few tweaks)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter(x=date_list, y=list_of_float_value, mode="markers+lines"))

scatter = fig.data[0]

def update_point(trace, points, selector):
  print("Sleeping")
scatter.on_click(update_point)

fig.show(renderer="colab")